<a href="https://colab.research.google.com/github/whyeon92/ESAA_OB/blob/Code_Study/ESAA_HW0927.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8.1 텍스트 분석 이해  

**[진행 프로세스]**
 1. 텍스트 사전 준비 작업, 전처리  
   : 텍스트를 피처로 만들기 전 클렌징, 대/소문자 변경, 특수문자 삭제, 단어 토큰화, 의미 없는 단어(Stop Word) 제거, 어근 추출 등의 텍스트 정규화 작업
 2. 피처/벡터화 추출  
   : 가공된 텍스트에서 피처 추출, 해당 피처에 벡터 값 할당   
    → BOW, Word2Vec 이용
 3. ML 모델 수립 및 학습/예측/평가  
   : 피처 벡터화된 데이터 세트에 머신러닝 모델 적용

# 8.2 텍스트 사전 준비 작업(텍스트 전처리) - 텍스트 정규화

문장 토큰화

In [1]:
from nltk import sent_tokenize
import nltk
nltk.download('punkt')

text_sample = 'The Matrix is everywhere its all around us, here even in this room. \
               You can see it out your window or on your television. \
               You feel it when you go to work, or go to church or pay your taxes.'
sentences = sent_tokenize(text=text_sample)
print(type(sentences),len(sentences))
print(sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


<class 'list'> 3
['The Matrix is everywhere its all around us, here even in this room.', 'You can see it out your window or on your television.', 'You feel it when you go to work, or go to church or pay your taxes.']


단어 토큰화

In [2]:
from nltk import word_tokenize

sentence = 'The Matrix is everywhere its all around us, here even in this room.'
words = word_tokenize(sentence)
print(type(words), len(words))
print(words)

<class 'list'> 15
['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.']


In [3]:
from nltk import word_tokenize, sent_tokenize

#여러개의 문장으로 된 입력 데이터를 문장별로 단어 토큰화 만드는 함수 생성
def tokenize_text(text):

    #문장별로 분리 토큰
    sentences = sent_tokenize(text)
    #분리된 문장별 단어 토큰화
    word_tokens = [word_tokenize(sentence) for sentence in sentences]
    return word_tokens

#여러 문장들에 대해 문장별 단어 토큰화 수행
word_tokens = tokenize_text(text_sample)
print(type(word_tokens),len(word_tokens))
print(word_tokens)

<class 'list'> 3
[['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.'], ['You', 'can', 'see', 'it', 'out', 'your', 'window', 'or', 'on', 'your', 'television', '.'], ['You', 'feel', 'it', 'when', 'you', 'go', 'to', 'work', ',', 'or', 'go', 'to', 'church', 'or', 'pay', 'your', 'taxes', '.']]


스탑워드 제거

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
print('영어 stop words 개수:',len(nltk.corpus.stopwords.words('english')))
print(nltk.corpus.stopwords.words('english')[:20])

영어 stop words 개수: 179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his']


In [6]:
import nltk

stopwords = nltk.corpus.stopwords.words('english')
all_tokens = []
#위 예제에서 3개의 문장별로 얻은 word_tokens list 에 대해 스톱 워드를 제거하는 반복문
for sentence in word_tokens:
    filtered_words=[]
    #개별 문장별로 토큰화된 문장 list에 대해 스톱 워드를 제거하는 반복문
    for word in sentence:
        #소문자로 모두 변환
        word = word.lower()
        #토큰화된 개별 단어가 스톱 워드의 단어에 포함되지 않으면 word_tokens에 추가
        if word not in stopwords:
            filtered_words.append(word)
    all_tokens.append(filtered_words)

print(all_tokens)

[['matrix', 'everywhere', 'around', 'us', ',', 'even', 'room', '.'], ['see', 'window', 'television', '.'], ['feel', 'go', 'work', ',', 'go', 'church', 'pay', 'taxes', '.']]


Stemming과 Lemmatization

In [7]:
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer()

print(stemmer.stem('working'),stemmer.stem('works'),stemmer.stem('worked'))
print(stemmer.stem('amusing'),stemmer.stem('amuses'),stemmer.stem('amused'))
print(stemmer.stem('happier'),stemmer.stem('happiest'))
print(stemmer.stem('fancier'),stemmer.stem('fanciest'))

work work work
amus amus amus
happy happiest
fant fanciest


In [8]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

lemma = WordNetLemmatizer()
print(lemma.lemmatize('amusing','v'),lemma.lemmatize('amuses','v'),lemma.lemmatize('amused','v'))
print(lemma.lemmatize('happier','a'),lemma.lemmatize('happiest','a'))
print(lemma.lemmatize('fancier','a'),lemma.lemmatize('fanciest','a'))

[nltk_data] Downloading package wordnet to /root/nltk_data...


amuse amuse amuse
happy happy
fancy fancy


**[클렌징]**  
 텍스트에서 분석에 오히려 방해가 되는 불필요한 문자들을 사전에 제거

**[토큰화]**  
 텍스트를 단위에 따라 나누는 작업.
 - 문장 토큰화 : 문장의 마침표, 개행문자 등 문장의 마지막을 뜻하는 기호로 문장을 나누기
 - 단어 토큰화 : 문장을 단어로 토큰화. 기본적으로 공백을 단위로 단어를 분리  

⚠️ 문장을 단어별로 하나씩 토큰화할 경우 문맥적 의미가 사라짐!  
→ n-gram으로 해결  
 n-gram: 단어를 하나하나 토큰화하는 게 아닌 n개의 단어를 하나의 토큰으로 생각하여 토큰화!

**[스톱워드 제거]**  
 스톱워드: 분석에 큰 의미를 갖지 않는 be동사, 관사 등의 단어  
 즉, 필수 문법적 요소지만 문맥적으로 큰 의미가 없는 단어  
 오히려 중요한 단어로 인지되면 안되는데 자주 나와서 그렇게 해석될 수 있기에 제거 필수  
 → nltk의 스톱워드 목록에서 필터링!

**[Stemming / Lemmatization]**  
 둘 다 단어의 원형인 어근을 매핑  
- Stemming  
 : 일반적인 방법, 더 단순화된 방법을 사용해서 철자가 훼손된 경우가 잦음!
- Lemmatization
 : 문법적 요소와 더 의미적인 부분을 찾아줘서 오랜시간이 걸리지만 더 깔끔한 처리

# 8.3 Bag of Words - BOW

In [9]:
import numpy as np

dense = np.array([[3, 0, 1], [0, 2, 0 ]])

In [10]:
from scipy import sparse

#0이 아닌 데이터 추출
data = np.array([3,1,2])

#행 위치와 열 위치를 각각 배열로 생성
row_pos = np.array([0,0,1])
col_pos = np.array([0,2,1])

#sparse 패키지의 coo_matrix를 이용하여 COO 형식으로 희소 행렬 생성
sparse_coo = sparse.coo_matrix((data, (row_pos,col_pos)))

In [11]:
sparse_coo.toarray()

array([[3, 0, 1],
       [0, 2, 0]])

In [12]:
from scipy import sparse

dense2 = np.array([[0,0,1,0,0,5],
                    [1,4,0,3,2,5],
                    [0,6,0,3,0,0],
                    [2,0,0,0,0,0],
                    [0,0,0,7,0,8],
                    [1,0,0,0,0,0]])

#0이 아닌 데이터 추출
data2 = np.array([1, 5, 1, 4, 3, 2, 5, 6, 3, 2, 7, 8, 1])

#행 위치와 열 위치를 각각 array로 생성
row_pos = np.array([0, 0, 1, 1, 1, 1, 1, 2, 2, 3, 4, 4, 5])
col_pos = np.array([2, 5, 0, 1, 3, 4, 5, 1, 3, 0, 3, 5, 0])

#COO 형식으로 변환
sparse_coo = sparse.coo_matrix((data2, (row_pos,col_pos)))

#행 위치 배열의 고유한 값들의 시작 위치 인덱스를 배열로 생성
row_pos_ind = np.array([0, 2, 7, 9, 10, 12, 13])

#CSR 형식으로 변환
sparse_csr = sparse.csr_matrix((data2, col_pos, row_pos_ind))

print('COO 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인')
print(sparse_coo.toarray())
print('CSR 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인')
print(sparse_csr.toarray())

COO 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인
[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]
CSR 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인
[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]


In [13]:
dense3 = np.array([[0,0,1,0,0,5],
             [1,4,0,3,2,5],
             [0,6,0,3,0,0],
             [2,0,0,0,0,0],
             [0,0,0,7,0,8],
             [1,0,0,0,0,0]])

coo = sparse.coo_matrix(dense3)
csr = sparse.csr_matrix(dense3)

**[Bag of Words]**  
: 문서가 가지는 모든 단어를 문맥, 순서를 무시하고 일괄적으로 단어에 대해 빈도 값을 부여해 피처 값을 추출하는 모델  

 - 🥰쉽고 빠르게 구축 가능, 그냥 간단히 단어 토큰화하고 횟수 세기
 - 🥰활용도가 높음

 - 🤢문맥 의미를 너무 반영 못함
 - 🤢희소 행렬 문제  
    : 문서마다 서로 다른 단어로 구성되어서 BOW의 단어 중 해당 문서에 사용되지 않는 단어가 더 많을 수 있음. 그런 경우 행렬이 대부분의 값이 0으로 채워지는 희소행렬이 되는데, 이는 일반적으로 머신러닝 알고리즘의 수행시간과 예측성능을 떨어뜨리기 때문에 나쁘다!  

**[벡터 피처화]**  
 BOW에서 벡터 피처화 → 모든 문서에서 모든 단어를 칼럼 형태로 나열하고 각 단어의 횟수(또는 정규화된 빈도)를 값으로 부여하는 데이터 세트 모델로 변경 → M개의 텍스트문서와 N개의 단어 → M*N형태의 행렬 구성  
 - 카운트 기반 벡터화  
  → 카운트 횟수가 많으면 중요하다! 그러나 이럴 경우 문서의 특징을 나타내기 보단 언어의 특성상 문장에 자주 사용될 수 밖에 없는 단어도 높은 값이 부여될 가능성이 존재.
 - TF-IDF 기반 벡터화  
   → 개별 문서에서 자주 나타나는 단어에 높은 가중치, 그러나 모든 문서에 자주 나타나는 단어엔 페널티를 줘서 위의 카운트 기반에서 나타나는 단점을 어느정도 완화.